## Discrete Dynamic Programming
A discrete dynamic program consists of the following components:

* finite set of states $S = \{0, \ldots, n-1\}$;
* finite set of available actions $A(s)$ for each state $s
\in S$ with $A = \bigcup_{s \in S} A(s) = \{0, \ldots, m-1\}$,
where $\mathit{SA} = \{(s, a) \in S \times A \mid a \in A(s)\}$
is the set of feasible state-action pairs;
* reward function $r\colon \mathit{SA} \to \mathbb{R}$, where
$r(s, a)$ is the reward when the current state is $s$ and
the action chosen is $a$;
* transition probability function $q\colon \mathit{SA} \to
\Delta(S)$, where $q(s'|s, a)$ is the probability that the state
in the next period is $s'$ when the current state is $s$
and the action chosen is $a$; and
* discount factor $0 \leq \gamma < 1$.

#### Policy Function $\pi$
For a policy function $\pi$, let $r_{\pi}$ and
$P_{\pi}$ be the reward vector and the transition probability
matrix for $\pi$, which are defined by $r_{\pi}(s) =
r(s, \pi(s))$ and $P_{\pi}(s, s') = q(s'|s, \pi(s))$,
respectively. The policy state value function $v_{\pi}$ for
$\pi$ is defined by



 $v_{\pi}(s) = \sum_{t=0}^{\infty}
                  \gamma^t (P_{\pi}^t r_{\pi})(s)
                  \quad (s \in S).$

The *optimal value function* $v^*$ is the function such that
$v^*(s) = \max_{\sigma} v_{\sigma}(s)$ for all $s \in S$. A
policy function $\sigma^*$ is *optimal* if $v_{\sigma^*}(s)
= v^*(s)$ for all $s \in S$.

The *Bellman equation* is written as



$v(s) = \max_{a \in A(s)} (r(s, a)
         + \beta \sum_{s' \in S} q(s'|s, a) v(s')) \quad(s \in S).$

The *Bellman operator* $T$ is defined by the right hand side of
the Bellman equation:


$(T v)(s) = \max_{a \in A(s)} (r(s, a)
             + \beta \sum_{s' \in S} q(s'|s, a) v(s')) \quad (s \in S).$

For a policy function $\sigma$, the operator $T_{\sigma}$ is
defined by


$(T_{\sigma} v)(s) = r(s, \sigma(s))
                      + \beta \sum_{s' \in S} q(s'|s, \sigma(s)) v(s')
                      \quad (s \in S),$

or $T_{\sigma} v = r_{\sigma} + \beta Q_{\sigma} v$.

The main result of the theory of dynamic programming states that the
optimal value function $v^*$ is the unique solution to the Bellman
equation, or the unique fixed point of the Bellman operator, and that
$\sigma^*$ is an optimal policy function if and only if it is
$v^*$-greedy, i.e., it satisfies $T v^* = T_{\sigma^*} v^*$.

Solution Algorithms
-------------------

The $DiscreteDP$ class currently implements the following solution
algorithms:

* value iteration;
* policy iteration;
* modified policy iteration.

Policy iteration computes an exact optimal policy in finitely many
iterations, while value iteration and modified policy iteration return
an $\varepsilon$-optimal policy and an
$\varepsilon/2$-approximation of the optimal value function for a
prespecified value of $\varepsilon$.

Our implementations of value iteration and modified policy iteration
employ the norm-based and span-based termination rules, respectively.

* Value iteration is terminated when the condition $\lVert T v - v
\rVert < [(1 - \beta) / (2\beta)] \varepsilon$ is satisfied.

* Modified policy iteration is terminated when the condition
$\mathrm{span}(T v - v) < [(1 - \beta) / \beta] \varepsilon$ is
satisfied, where $\mathrm{span}(z) = \max(z) - \min(z)$.

## Early Abortion

- In-place dp
-  Sweeping
    - state which changes most
- Real-time

## sampling

In [1]:
from quantecon.markov import DiscreteDP

In [2]:
R = [[5, 10], [-1, -float('inf')]]
Q = [[(0.5, 0.5), (0, 1)], [(0, 1), (0.5, 0.5)]]
beta = 0.95
ddp = DiscreteDP(R, Q, beta)

In [9]:
import pprint
pp=pprint.PrettyPrinter(indent=4)

In [21]:
import numpy as np
R_array=np.asarray(R)
Q_array=np.asarray(Q)
pp.pprint(R_array)
pp.pprint(Q_array)

array([[  5.,  10.],
       [ -1., -inf]])
array([[[0.5, 0.5],
        [0. , 1. ]],

       [[0. , 1. ],
        [0.5, 0.5]]])


### Policy iteration

In [16]:
res = ddp.solve(method='policy_iteration', v_init=[0, 0])

In [18]:
print(res.sigma)
print(res.v)

[0 0]
[ -8.57142857 -20.        ]


### Value iteration

In [19]:
res_v = ddp.solve(method='value_iteration', v_init=[0, 0])

In [20]:
print(res_v.sigma)
print(res_v.v)

[0 0]
[ -8.570939   -19.99951043]


值迭代策略利用了上节中公式（2）

    内循环的实现有两种策略：

    1、 同步迭代法

    拿初始化后的第一次迭代来说吧，初始状态所有的V(s)都为0。然后对所有的s都计算新的V(s)=R(s)+0=R(s)。在计算每一个状态时，得到新的V(s)后，先存下来，不立即更新。待所有的s的新值V(s)都计算完毕后，再统一更新。

这样，第一次迭代后，V(s)=R(s)。

    2、 异步迭代法

    与同步迭代对应的就是异步迭代了，对每一个状态s，得到新的V(s)后，不存储，直接更新。这样，第一次迭代后，大部分V(s)>R(s)。

    不管使用这两种的哪一种，最终V(s)会收敛到V*(s)。知道了V*后，我们再使用公式（3）来求出相应的最优策略clip_image069[4]，当然clip_image069[5]可以在求V*的过程中求出。

    * 策略迭代法